In [31]:
import socket
import time
import select

class AMDC:
    def __init__(self, 
                 amdc_ip_addr = '192.168.1.10',
                 amdc_ip_port = 7):
        # Serial port configuration
        self.amdc_ip_addr = amdc_ip_addr
        self.amdc_ip_port = amdc_ip_port
        
        self.sockets = []

    def new_socket(self, socket_type):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((self.amdc_ip_addr, self.amdc_ip_port))
        
        s.setblocking(1)
        
        if socket_type in ['ascii_cmd']:
            s.send(bytearray([12, 34]))
        elif socket_type in ['log_var']:
            s.send(bytearray([56, 78]))
        else:
            raise NameError('InvalidSocketType')
        
        self.sockets.append(s)
        return s
    
    def close_all_sockets(self):
        for s in self.sockets:
            s.close()
    
    @staticmethod
    def cmd(socket, cmd_str):
        cmd_bytes = str.encode(cmd_str)
        socket.send(cmd_bytes)
        return len(cmd_bytes)
    
    @staticmethod
    def cmdresp(socket, cmd_str, timeout_sec=0.3):
        socket.send(str.encode(cmd_str))
        
        retdata = []
        
        socket.setblocking(0)
        
        loops = 5
        for i in range(0,loops):
            ready = select.select([socket], [], [], timeout_sec/loops)
            if ready[0]:
                retdata.append(socket.recv(4096))
        
        socket.setblocking(1)
        
        return retdata

In [32]:
amdc = AMDC()

In [33]:
amdc.close_all_sockets()

In [34]:
s1 = amdc.new_socket('ascii_cmd')

In [35]:
AMDC.cmd(s1, 'blink init\n')

11

In [37]:
data = AMDC.cmdresp(s1, 'blink init\n')
data

[b'blink init\r\nFAILURE\r\n\n']

In [19]:
# Ha, we can run this and the AMDC does not miss a BEAT.
# 
# The host core is buffering all the incoming TCP packet
# data and then the ICC code is slowly pushing the data
# to the other core, 1 byte every 100 usec...
#
# This is awesome :)

num_loops = 1000
expected_success = 1*num_loops

bytes_sent = 0

for i in range(0,num_loops):
    bytes_sent = bytes_sent + AMDC.cmd(s1, 'cnt inc\n')
#     bytes_sent = bytes_sent + AMDC.cmd(s1, 'blink deinit\n')
#     bytes_sent = bytes_sent + AMDC.cmd(s1, 'hw led set 0 255 0 0\n')
#     bytes_sent = bytes_sent + AMDC.cmd(s1, 'hw led set 1 0 255 0\n')
#     bytes_sent = bytes_sent + AMDC.cmd(s1, 'hw led set 2 0 0 255\n')
#     bytes_sent = bytes_sent + AMDC.cmd(s1, 'hw led set 3 255 255 255\n')
#     bytes_sent = bytes_sent + AMDC.cmd(s1, 'blink init\n')
    
print("expected_success:", expected_success)
print("bytes_sent:", bytes_sent)

expected_success: 1000
bytes_sent: 8000


In [173]:
retdata = []

s1.setblocking(0)

timeout_sec = 0.5
loops = 10
for i in range(0,loops):
    ready = select.select([s1], [], [], timeout_sec/loops)
    if ready[0]:
        retdata.append(s1.recv(10*1024*1024))

s1.setblocking(1)

if len(retdata) == 1:
    print("num data:", len(retdata[0]))
else:
    print("no data")

num data: 16878


In [174]:
retdata[0].decode("utf-8").count("SUCCESS")

888

In [175]:
retdata[0].decode("utf-8").count("FAILURE")

0

In [176]:
retdata[0].decode("utf-8").count("INVALID")

0

In [177]:
retdata[0].decode("utf-8").count("UNKNOWN")

0

In [178]:
retdata[0].decode("utf-8").split('\n')

['cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\r',
 'SUCCESS\r',
 '',
 'cnt inc\

In [179]:
data = AMDC.cmdresp(s1, 'cnt get\n')
data

[b'cnt get\r\nCNT = 1000\r\nSUCCESS\r\n\n']

In [180]:
data = AMDC.cmdresp(s1, 'cnt clr\n')
data

[b'cnt clr\r\nSUCCESS\r\n\n']

In [181]:
data = AMDC.cmdresp(s1, 'cnt get\n')
data

[b'cnt get\r\nCNT = 0\r\nSUCCESS\r\n\n']

# Test Out Logger

In [38]:
import AMDC_Logger as AMDC_Logger

In [44]:
mfpath = AMDC_Logger.find_mapfile(r'C:\Users\Nathan\Documents\GitHub\AMDC-Firmware\sdk\app_cpu1')
mapfile = AMDC_Logger.Mapfile(mfpath)
addr = int(mapfile.address('LOG_vsi_a'), 0)
addr

537526536

In [40]:
mapfile.address('LOG_vsi_b')

'0x00000000200a0110'

In [83]:
data = AMDC.cmdresp(s1, 'log info\n')
data[0].decode("utf-8").split('\n')

['log info\r',
 'SUCCESS\r',
 '',
 'Log Info\r',
 '--------\r',
 'Max slots: 32\r',
 'Max sample depth: 100000\r',
 'Max sample rate: 10000 Hz\r',
 '--------\r',
 'Slot 0:\r',
 '  Name: LOG_vsi_a\r',
 '  Type: double\r',
 '  Memory address: 0x200A0108\r',
 '  Sampling interval (usec): 10000\r',
 '  Num samples: 221\r',
 'Slot 1: unused\r',
 'Slot 2: unused\r',
 'Slot 3: unused\r',
 'Slot 4: unused\r',
 'Slot 5: unused\r',
 'Slot 6: unused\r',
 'Slot 7: unused\r',
 'Slot 8: unused\r',
 'Slot 9: unused\r',
 'Slot 10: unused\r',
 'Slot 11: unused\r',
 'Slot 12: unused\r',
 'Slot 13: unused\r',
 'Slot 14: unused\r',
 'Slot 15: unused\r',
 'Slot 16: unused\r',
 'Slot 17: unused\r',
 'Slot 18: unused\r',
 'Slot 19: unused\r',
 'Slot 20: unused\r',
 'Slot 21: unused\r',
 'Slot 22: unused\r',
 'Slot 23: unused\r',
 'Slot 24: unused\r',
 'Slot 25: unused\r',
 'Slot 26: unused\r',
 'Slot 27: unused\r',
 'Slot 28: unused\r',
 'Slot 29: unused\r',
 'Slot 30: unused\r',
 'Slot 31: unused\r',
 '\r',

In [46]:
data = AMDC.cmdresp(s1, 'log reg 0 LOG_vsi_a {} 100 double\n'.format(addr))
print(data[0].decode("utf-8").split('\n'))

['log reg 0 LOG_vsi_a 537526536 100 double\r', 'SUCCESS\r', '', '']


In [63]:
data = AMDC.cmdresp(s1, 'log start')
print(data[0].decode("utf-8").split('\n'))

time.sleep(0.1)

data = AMDC.cmdresp(s1, 'log stop')
print(data[0].decode("utf-8").split('\n'))

IndexError: list index out of range

In [124]:
AMDC.cmd(s1, 'log dump text 0\n')

16

In [125]:
retdata = []

s1.setblocking(0)

timeout_sec = 2.0
loops = 2
for i in range(0,loops):
    ready = select.select([s1], [], [], timeout_sec/loops)
    if ready[0]:
        retdata.append(s1.recv(10*1024*1024))

s1.setblocking(1)

if len(retdata) >= 1:
    print("num data:", len(retdata[0]))
else:
    print("no data")

num data: 5725


In [126]:
retdata

[b"log dump text 0\r\nLOG OF VARIABLE: 'LOG_vsi_a'\r\nNUM SAMPLES: 221\r\n-------START-------\r\n> 68772000\t\t-7068.679199\r\n> 68782000\t\t-7283.622070\r\n> 68792000\t\t-7425.789551\r\n> 68802000\t\t-7493.761230\r\n> 68812000\t\t-7486.857422\r\n> 68822000\t\t-7405.147461\r\n> 68832000\t\t-7249.447754\r\n> 68842000\t\t-7021.313965\r\n> 68852000\t\t-6723.025879\r\n> 68862000\t\t-6357.562988\r\n> 68872000\t\t-5928.577637\r\n> 68882000\t\t-5440.355957\r\n> 68892000\t\t-4897.776367\r\n> 68902000\t\t-4306.259277\r\n> 68912000\t\t-3671.715576\r\n> 68922000\t\t-3000.485352\r\n> 68932000\t\t-2299.275391\r\n> 68942000\t\t-1575.091675\r\n> 68952000\t\t-835.170227\r\n> 68962000\t\t-86.904030\r\n> 68972000\t\t662.230469\r\n> 68982000\t\t1404.748169\r\n> 68992000\t\t2133.230225\r\n> 69002000\t\t2840.397705\r\n> 69012000\t\t3519.184570\r\n> 69022000\t\t4162.809082\r\n> 69032000\t\t4764.840332\r\n> 69042000\t\t5319.262695\r\n> 69052000\t\t5820.537109\r\n> 69062000\t\t6263.654297\r\n> 69072000\t\t664

# Test AMDC Class

In [1]:
from AMDC import AMDC

In [2]:
amdc = AMDC()
amdc.setup_comm_defaults('uart')
amdc.uart_init('COM6')

In [2]:
amdc = AMDC()
amdc.setup_comm_defaults('eth')
amdc.eth_init()
s1 = amdc.eth_new_socket('ascii_cmd')
amdc.eth_set_default_ascii_cmd_socket(s1)

In [10]:
amdc.connect()
amdc.cmd('hw anlg read 0')
amdc.cmd('hw anlg read 1')
amdc.disconnect()

	> hw anlg read 0
['hw anlg read 0', '0.005000V', 'SUCCESS']
	> hw anlg read 1
['hw anlg read 1', '-0.012500V', 'SUCCESS']


In [3]:
amdc.connect()

In [45]:
for j in range(0,10):
    for i in range(0,8):
        resp = amdc.cmd('hw anlg read {}'.format(i))

	> hw anlg read 0
['hw anlg read 0', '0.005000V', 'SUCCESS']
	> hw anlg read 1
['hw anlg read 1', '-0.012500V', 'SUCCESS']
	> hw anlg read 2
['hw anlg read 2', '-0.005000V', 'SUCCESS']
	> hw anlg read 3
['hw anlg read 3', '-0.010000V', 'SUCCESS']
	> hw anlg read 4
['hw anlg read 4', '-0.005000V', 'SUCCESS']
	> hw anlg read 5
['hw anlg read 5', '-0.015000V', 'SUCCESS']
	> hw anlg read 6
['hw anlg read 6', '10.005000V', 'SUCCESS']
	> hw anlg read 7
['hw anlg read 7', '-10.007500V', 'SUCCESS']
	> hw anlg read 0
['hw anlg read 0', '0.002500V', 'SUCCESS']
	> hw anlg read 1
['hw anlg read 1', '-0.012500V', 'SUCCESS']
	> hw anlg read 2
['hw anlg read 2', '-0.005000V', 'SUCCESS']
	> hw anlg read 3
['hw anlg read 3', '-0.012500V', 'SUCCESS']
	> hw anlg read 4
['hw anlg read 4', '-0.007500V', 'SUCCESS']
	> hw anlg read 5
['hw anlg read 5', '-0.015000V', 'SUCCESS']
	> hw anlg read 6
['hw anlg read 6', '10.002500V', 'SUCCESS']
	> hw anlg read 7
['hw anlg read 7', '-10.010000V', 'SUCCESS']
	> hw an

In [6]:
amdc.cmd('hw anlg read 0')

	> hw anlg read 0
['hw anlg read 0', '0.005000V', 'SUCCESS']


['hw anlg read 0', '0.005000V', 'SUCCESS']

In [4]:
amdc.cmd('1234')
amdc.cmd('2345')
amdc.cmd('3456')
amdc.cmd('4567')
amdc.cmd('5678')
amdc.cmd('6789')
amdc.cmd('7890')
;

	> 1234
['1234', 'UNKNOWN CMD']
	> 2345
['2345', 'UNKNOWN CMD']
	> 3456
['3456', 'UNKNOWN CMD']
	> 4567
['4567', 'UNKNOWN CMD']
	> 5678
['5678', 'UNKNOWN CMD']
	> 6789
['6789', 'UNKNOWN CMD']
	> 7890
['7890', 'UNKNOWN CMD']


''

In [5]:
for j in range(0,10):
    resp = amdc.cmd('counter inc')
    resp = amdc.cmd('counter get')

	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 11', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 12', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 13', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 14', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 15', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 16', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 17', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 18', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 19', 'SUCCESS']
	> counter inc
['counter inc', 'SUCCESS']
	> counter get
['counter get', 'CNT = 20', 'SUCCESS']
